In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Specify the folder where the CIFAR-10 batch files are
cifar10_dir = '/kaggle/input/deep-learning-mini-project-spring-24-nyu/cifar-10-python/cifar-10-batches-py'

# Load the label names
meta_data_dict = load_cifar_batch(os.path.join(cifar10_dir, 'batches.meta'))
label_names = meta_data_dict[b'label_names']

# Load one batch for demonstration (e.g., data_batch_1)
batch_1_dict = load_cifar_batch(os.path.join(cifar10_dir, 'data_batch_1'))
train_images = batch_1_dict[b'data']
train_labels = batch_1_dict[b'labels']

# Reshape the images
train_images = train_images.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)

# Display the first 10 images and labels
plt.figure(figsize=(20, 4))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(train_images[i])
    plt.title(label_names[train_labels[i]].decode('utf-8'))  # Decoding from bytes to string
    plt.axis('off')
plt.show()


In [ ]:
# Read the test file, note that it has no labels and needs to be used with your model inference to predict outputs.

def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    return batch

# Load the batch
cifar10_batch = load_cifar_batch('/kaggle/input/deep-learning-mini-project-spring-24-nyu/cifar_test_nolabels.pkl')

# Extract images
images = cifar10_batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1) 

In [ ]:
# Extract images and convert back to image format
images = cifar10_batch[b'data']
# Reshape to (number of images, width, height, channels)
images = images.reshape((-1, 3, 32, 32)).transpose(0, 2, 3, 1)

# Display the first 10 images
plt.figure(figsize=(20, 4))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(images[i])
    plt.axis('off')
plt.show()

## teacher - widenet

In [ ]:
import torchvision.models as models
teacher = models.wide_resnet101_2(pretrained=False)
teacher.fc = nn.Linear(teacher.fc.in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher = teacher.to(device)  

In [ ]:
total_params_net = "{:,}".format(sum(p.numel() for p in teacher.parameters()))
print(f"wide net parameters: {total_params_net}")

In [ ]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
# Step 1: Load and preprocess the CIFAR-10 dataset
train_trans=transforms.Compose([transforms.ToTensor(),
           transforms.RandomCrop(32, padding=4),
           transforms.RandomHorizontalFlip(),
           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_trans=transforms.Compose([transforms.ToTensor(),
           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_trans)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_trans)
testloader = DataLoader(testset, batch_size=int(128/4), shuffle=False, num_workers=16)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Step 2: Define the ResNet model
#net = ResNet(BasicBlock, [2,2,2,2])  
teacher = teacher.to('cuda' if torch.cuda.is_available() else 'cpu')

# Step 3: Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(teacher.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
#optimizer = optim.Adam(teacher.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
best_acc = 0.0
# Step 4: Train the model
for epoch in range(200):  # number of epochs to train for
    teacher.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        optimizer.zero_grad()

        outputs = teacher(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print('[Epoch: %d, Mini-batch: %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    scheduler.step()

    # Optional: Print accuracy on the test set after each epoch
    correct = 0
    total = 0
    with torch.no_grad():
        teacher.eval()
        for data in testloader:
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = teacher(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100.0*correct/total
    if(acc>best_acc):
        #only save the best model
        best_acc = acc
        PATH = './best_teacher.pth'
        torch.save(teacher.state_dict(), PATH)
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

print('Finished Training')

## ResNet Architecture 

In [ ]:
'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        # F_i: Convolutional kernel size for BasicBlock can be changed in the conv1 and conv2 lines
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        # K_i: Skip connection kernel size can be changed here if the condition is met
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        # B_i: Number of blocks in each layer is set here through the num_blocks parameter
        # C_i: Number of channels in each layer is set through the planes parameter
        self.layer1 = self._make_layer(block, 100, num_blocks[0], stride=1) #64 #best64
        self.layer2 = self._make_layer(block, 150, num_blocks[1], stride=2) #128 #best128
        self.layer3 = self._make_layer(block, 215, num_blocks[2], stride=2) #256 #best160
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2) #512 #best256
        self.linear = nn.Linear(256*block.expansion, num_classes) #512 #best256

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # P: Average pool kernel size is set here
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [ ]:
net=ResNet(BasicBlock, [2,2, 2, 2]).cuda()

In [ ]:
total_params_net = "{:,}".format(sum(p.numel() for p in net.parameters()))
print(f"modified resnet parameters: {total_params_net}")

## Training & Testing

In [ ]:
teacher = models.wide_resnet101_2(pretrained=False)
teacher.fc = nn.Linear(teacher.fc.in_features, 10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher = teacher.to(device)  
teacher.load_state_dict(torch.load('./best_teacher.pth',map_location=device))
teacher.to(device)
teacher.eval()

In [ ]:
def distillation_loss(y_student, y_teacher, y_true, T=2.0, alpha=0.7):
    KL_div = nn.KLDivLoss()(F.log_softmax(y_student/T, dim=1),
                            F.softmax(y_teacher/T, dim=1)) * (alpha * T * T)
    CE_loss = nn.CrossEntropyLoss()(y_student, y_true) * (1. - alpha)
    return KL_div + CE_loss

In [ ]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Step 1: Load and preprocess the CIFAR-10 dataset
train_trans=transforms.Compose([transforms.ToTensor(),
           transforms.RandomCrop(32, padding=4),
           transforms.RandomHorizontalFlip(),
           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_trans=transforms.Compose([transforms.ToTensor(),
           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_trans)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_trans)
testloader = DataLoader(testset, batch_size=int(128/4), shuffle=False, num_workers=16)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Step 2: Define the ResNet model
#net = ResNet(BasicBlock, [2,2,2,2])  
net = net.to('cuda' if torch.cuda.is_available() else 'cpu')

# Step 3: Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
optimizer = optim.SGD(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
best_acc = 0.0
teacher.eval()
# Step 4: Train the model
for epoch in range(200):  # number of epochs to train for
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        optimizer.zero_grad()
        with torch.no_grad():
            teacher_outputs = teacher(inputs)
        outputs = net(inputs)
        loss = distillation_loss(outputs, teacher_outputs, labels)
        #loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print('[Epoch: %d, Mini-batch: %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    scheduler.step()

    # Optional: Print accuracy on the test set after each epoch
    correct = 0
    total = 0
    with torch.no_grad():
        net.eval()
        for data in testloader:
            images, labels = data
            images, labels = images.to('cuda'), labels.to('cuda')
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100.0*correct/total
    if(acc>best_acc):
        #only save the best model
        best_acc = acc
        PATH = './cifar_net.pth'
        torch.save(net.state_dict(), PATH)
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

print('Finished Training')

In [ ]:
import pickle
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import torchvision

def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        batch = pickle.load(fo, encoding='bytes')
    return batch[b'data']  # Assuming the data is stored under this key

# Preprocess and load the batch
batch_data = load_cifar_batch('/kaggle/input/deep-learning-mini-project-spring-24-nyu/cifar_test_nolabels.pkl')
batch_data = batch_data.reshape((-1, 3, 32, 32))  # Reshape to match the CIFAR-10 format
batch_data = batch_data / 255.0  # Normalize to [0, 1]

# Apply the same transformations as for the test set
transform = transforms.Compose([
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Convert numpy array to tensor
tensor_data = torch.tensor(batch_data, dtype=torch.float32)
transformed_data = torch.stack([transform(img) for img in tensor_data])

# Create a DataLoader
dataset = TensorDataset(transformed_data)
dataloader = DataLoader(dataset, batch_size=100, shuffle=False)

#Load the best model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model = ResNet(BasicBlock, [2,2,2,2]).cuda()
best_model.load_state_dict(torch.load('./cifar_net.pth',map_location=device))
best_model.to(device)
best_model.eval()

# Inference
# Inference with ID and prediction
predictions_with_id = []
with torch.no_grad():
    for batch_idx, data in enumerate(dataloader):
        inputs = data[0].cuda()
        outputs = best_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        batch_predictions = predicted.cpu().numpy()

        # Add ID and prediction to the list
        start_id = batch_idx * dataloader.batch_size
        for i, pred in enumerate(batch_predictions):
            predictions_with_id.append((start_id + i, int(pred)))

# predictions_with_id now contains tuples of (image_id, predicted_class)


In [ ]:
import csv

csv_file_path = 'cifar_predictions.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["ID", "Labels"])  # Writing header
    writer.writerows(predictions_with_id)

